In [1]:
import pandas as pd

In [2]:
s3_prefix = "s3://aegovan-data/pubmed_asbtract/predictions_multi_95/"

In [3]:
local_temp_dir = "./temp/pred_results"

In [4]:
!rm -rf $local_temp_dir
!mkdir -p $local_temp_dir

In [5]:
#!aws s3 cp s3://aegovan-data/pubmed_asbtract/predictions_multi_95/pubmed19n0538.json.txt.json.prediction.json .

In [6]:
import boto3
import glob
from multiprocessing.dummy import Pool as ThreadPool
import argparse
import datetime 
import os


def uploadfile(localpath, s3path):
        """
Uploads a file to s3
        :param localpath: The local path
        :param s3path: The s3 path in format s3://mybucket/mydir/mysample.txt
        """

        bucket, key = get_bucketname_key(s3path)

        if key.endswith("/"):
            key = "{}{}".format(key, os.path.basename(localpath))
        
        s3 = boto3.client('s3')
        
        s3.upload_file(localpath, bucket, key)

def get_bucketname_key(uripath):
    assert uripath.startswith("s3://")

    path_without_scheme = uripath[5:]
    bucket_end_index = path_without_scheme.find("/")

    bucket_name = path_without_scheme
    key = "/"
    if bucket_end_index > -1:
        bucket_name = path_without_scheme[0:bucket_end_index]
        key = path_without_scheme[bucket_end_index + 1:]

    return bucket_name, key


def download_file(s3path, local_dir):
    bucket, key = get_bucketname_key(s3path)
    
    s3 = boto3.client('s3')
    
    local_file = os.path.join(local_dir, s3path.split("/")[-1])
    

    s3.download_file(bucket, key, local_file)
    
def download_object(s3path):
    bucket, key = get_bucketname_key(s3path)
    
    s3 = boto3.client('s3')    

    s3_response_object = s3.get_object(Bucket=bucket, Key=key)
    object_content = s3_response_object['Body'].read()
    
    return len(object_content)



def list_files(s3path_prefix):
    assert s3path_prefix.startswith("s3://")
    assert s3path_prefix.endswith("/")
    
    bucket, key = get_bucketname_key(s3path_prefix)
    
   
   
    s3 = boto3.resource('s3')
    
    bucket = s3.Bucket(name=bucket)

    return ( (o.bucket_name, o.key) for o in bucket.objects.filter(Prefix=key))





def upload_files(local_dir, s3_prefix, num_threads=20):    
    input_tuples = ( (f,  s3_prefix) for f in glob.glob("{}/*".format(local_dir)))
    
    with ThreadPool(num_threads) as pool:
        pool.starmap(uploadfile, input_tuples)
    


def download_files(s3_prefix, local_dir, num_threads=20):    
    input_tuples = ( ("s3://{}/{}".format(s3_bucket,s3_key),  local_dir) for s3_bucket, s3_key in list_files(s3_prefix))
    
    with ThreadPool(num_threads) as pool:
        results = pool.starmap(download_file, input_tuples)
        
        

def download_objects(s3_prefix, num_threads=20):    
    s3_files = ( "s3://{}/{}".format(s3_bucket,s3_key) for s3_bucket, s3_key in list_files(s3_prefix))
    
    with ThreadPool(num_threads) as pool:
        results = pool.map(download_object, s3_files)
        
    return sum(results)/1024
        

def get_directory_size(start_path):
    total_size = 0
    for dirpath, dirnames, filenames in os.walk(start_path):
        for f in filenames:
            fp = os.path.join(dirpath, f)
            # skip if it is symbolic link
            if not os.path.islink(fp):
                total_size += os.path.getsize(fp)
    return total_size

def get_s3file_size(bucket, key):
    s3 = boto3.client('s3')
    response = s3.head_object(Bucket=bucket, Key=key)
    size = response['ContentLength']
    return size
    
def download_files_min_files(s3_prefix, local_dir, min_file_size=310, num_threads=20):    
    input_tuples = ( ("s3://{}/{}".format(s3_bucket,s3_key),  local_dir) for s3_bucket, s3_key in list_files(s3_prefix) if get_s3file_size(s3_bucket, s3_key) > min_file_size )
    
    with ThreadPool(num_threads) as pool:
        results = pool.starmap(download_file, input_tuples)
        


In [7]:
%%time

download_files_min_files(s3_prefix, local_temp_dir)

In [12]:
!ls -l $local_temp_dir | wc -l

     501


In [26]:
total = 0
largest_df = None
value_dict={}
for f in os.listdir(local_temp_dir):
    df = pd.read_json(os.path.join(local_temp_dir, f))
    if largest_df is None:
        largest_df = df
    
    if df.shape[0] > largest_df.shape[0]:
        largest_df=df
        
    prediction_counts = df["predicted"].value_counts().to_dict()
    for k,v in prediction_counts.items():      
        value_dict [k] =  v +  value_dict.get(k, 0)
    
    #total records
    total += df.shape[0]

In [28]:
value_dict

{'phosphorylation': 5626, 'dephosphorylation': 15}

In [27]:
total

5641

In [9]:

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 10000)
pd.set_option('display.max_rows', 100)

import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 12})

In [17]:
largest_df.shape

(59, 16)

In [30]:
largest_df[["abstract", "normalised_abstract", "participant1Id","participant2Id", "pubmedId", "predicted" ,"predicted_confidence" ]].sample(n=20)

,abstract,normalised_abstract,participant1Id,participant2Id,pubmedId,predicted,predicted_confidence
1445,"D-sites are a class of MAPK-docking sites that have been found in many MAPK regulators and substrates. A single functional, high affinity D-site has been identified near the N terminus of each of the MAPK kinases (MKKs or MEKs) MEK1, MEK2, MKK3, MKK4, and MKK6. Here we demonstrated that MKK7 recognizes its target JNK by a novel mechanism involving a partially cooperative interaction of three low affinity D-sites in the N-terminal domain of MKK7. Mutations of the conserved residues within any one of the three docking sites (D1, D2, and D3) disrupted the ability of the N-terminal domain of MKK7beta to bind JNK1 by about 50-70%. Moreover, mutation of any two of the three D-sites reduced binding by about 80-90%, and mutation of all three reduced binding by 95%. Full-length MKK7 containing combined D1/D2 mutations was compromised for binding to JNK1 and exhibited reduced JNK1 kinase activity when compared with wild-type MKK7. Peptide versions of the D-sites from MKK4 or the JIP-1 scaffold protein inhibited MKK7-JNK binding, suggesting that all three JNK regulators bind to the same region of JNK. Moreover, peptide versions of any of the three D-sites of MKK7 inhibited the ability of JNK1 and JNK2 to phosphorylate their transcription factor substrates c-Jun and ATF2, suggesting that D-site-containing substrates also compete with MKK7 for docking to JNK. Finally, MKK7-derived D-site peptides exhibited selective inhibition of JNK1 versus ERK2. We conclude that MKK7 contains three JNK-docking sites that interact to selectively bind JNK and contribute to JNK signal transmission and specificity.","D-sites are a class of MAPK-docking sites that have been found in many MAPK regulators and substrates. A single functional, high affinity D-site has been identified near the N terminus of each of the MAPK kinases (MKKs or MEKs) Q02750, P36507, P46734, P45985, and P52564. Here we demonstrated that O14733 recognizes its target P45983 by a novel mechanism involving a partially cooperative interaction of three low affinity D-sites in the N-terminal domain of O14733. Mutations of the conserved residues within any one of the three docking sites (D1, D2, and D3) disrupted the ability of the N-terminal domain of MKK7beta to bind P45983 by about 50-70%. Moreover, mutation of any two of the three D-sites reduced binding by about 80-90%, and mutation of all three reduced binding by 95%. Full-length O14733 containing combined D1/D2 mutations was compromised for binding to P45983 and exhibited reduced P45983 kinase activity when compared with wild-type O14733. Peptide versions of the D-sites from P45985 or the Q9UQF2 scaffold protein inhibited O14733-P45983 binding, suggesting that all three P45983 regulators bind to the same region of P45983. Moreover, peptide versions of any of the three D-sites of O14733 inhibited the ability of P45983 and P45984 to phosphorylate their transcription factor substrates P05412 and P15336, suggesting that D-site-containing substrates also compete with O14733 for docking to P45983. Finally, O14733-derived D-site peptides exhibited selective inhibition of P45983 versus P28482. We conclude that O14733 contains three P45983-docking sites that interact to selectively bind P45983 and contribute to P45983 signal transmission and specificity.",P45985,Q9UQF2,16533805,phosphorylation,0.965810
1431,"D-sites are a class of MAPK-docking sites that have been found in many MAPK regulators and substrates. A single functional, high affinity D-site has been identified near the N terminus of each of the MAPK kinases (MKKs or MEKs) MEK1, MEK2, MKK3, MKK4, and MKK6. Here we demonstrated that MKK7 recognizes its target JNK by a novel mechanism involving a partially cooperative interaction of three low affinity D-sites in the N-terminal domain of MKK7. Mutations of the conserved residues within any one of the three docking sites (D1, D2, and D3